# Assignment 2 : Mitigating Algorithmic Biases
Anukul K Singh <br>
Mar 11th 2024

In this assignment, we will explore practical strategies for mitigating algorithmic bias, focusing on the COMPAS dataset discussed in class. Your task is to complete the provided code blocks to ensure they function as expected. Carefully review each step in this notebook. When you encounter a placeholder marked with "###replace this with your own code!!!", replace it with the appropriate code.

## 1. Mitigating bias in recidivism risk prediction.

In this assignment, we'll utilize the COMPAS Dataset to predict recidivism, which refers to the likelihood of a convicted criminal to reoffend. These predictions can influence decisions regarding bail eligibility before a trial. The ProPublica investigation highlighted significant false positive rates for African American arrestees, demonstrating the algorithm's disparate impact.

In [24]:
#import libraries
import pandas as pd
import numpy as np
from itertools import product
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt


### 1.1 Clean Data

To replicate aspects of the original ProPublica analysis, we will clean the COMPAS Dataset, focusing on its use in recidivism prediction.

For our analysis, we will simplify the dataset to include only seven key features: three sensitive attributes (age, sex, and race) and four features related to current charges, prior charge counts, and arrest records.

As our label, we look at if the individual did actually commit another crime within a 2-year window of their arrest.

In [25]:
csv_url = 'https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv'

# Read the CSV file directly into a pandas DataFrame
raw_data = pd.read_csv(csv_url)
raw_data.head()

# From the original compas analysis
df = raw_data[((raw_data['days_b_screening_arrest'] <=30) &
      (raw_data['days_b_screening_arrest'] >= -30) &
      (raw_data['is_recid'] != -1) &
      (raw_data['c_charge_degree'] != 'O') &
      (raw_data['score_text'] != 'N/A')
     )]

print('Num rows after filtering: %d' % len(df))
print('Num with two-year recidivism: %d' % sum(df['two_year_recid'] == 1))

features = ['age', 'race', 'sex',
            'juv_fel_count', 'juv_misd_count', 'juv_other_count',
            'priors_count', 'c_charge_degree']

y = df['two_year_recid'].copy()
X = df[features].copy()

## For the purpose of this analysis, we want to examine the disparate impact on African Americans.
## We will group all non-African American arrestees into a single group (group=0)
## so that there are only two racial groups.

X['groups'] = np.where(X['race'] == 'African-American', 1, 0)
X = pd.get_dummies(X, columns = ['sex', 'c_charge_degree'])
X.drop(['race', 'sex_Female', 'c_charge_degree_F'], inplace = True, axis=1)
print('Features Matrix (below):')
X.head()

Num rows after filtering: 6172
Num with two-year recidivism: 2809
Features Matrix (below):


,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,groups,sex_Male,c_charge_degree_M
0,69,0,0,0,0,0,True,False
1,34,0,0,0,0,1,True,False
2,24,0,0,1,4,1,True,False
5,44,0,0,0,0,0,True,True
6,41,0,0,0,14,0,True,False


### 1.2 Split Data and Examine Recidivism Rates by Group

Split the dataset into training and testing subsets. Analyze the recidivism rates across different groups within the dataset to understand potential disparities.

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12345)

# Calculate the percentage of groups in the training and testing datasets
percent_aa_train = X_train['groups'].mean()
percent_others_train = 1 - percent_aa_train

percent_aa_test = X_test['groups'].mean()
percent_others_test = 1 - percent_aa_test

print('% African-American in train: ', percent_aa_train)
print('% Others in train: ', percent_others_train)

print('% African-American in test: ', percent_aa_test)
print('% Others in test: ', percent_others_test)

# Calculate the recidivism rate differences
def recidivism_rate_difference(X, y):
    # Create a DataFrame from the groups and labels to facilitate calculations
    data = pd.DataFrame({'Group': X['groups'], 'Recidivism': y})
    # Calculate recidivism rates by group
    rates = data.groupby('Group')['Recidivism'].mean()
    # Difference between the groups
    return abs(rates.diff().iloc[-1])

diff_train = recidivism_rate_difference(X_train, y_train)
diff_test = recidivism_rate_difference(X_test, y_test)

print('Difference in Recidivism Rate between Groups in Training Data: ',
     diff_train)

print('Difference in Recidivism Rate between Groups in Test Data: ',
     diff_test)

% African-American in train:  0.5124569576665992
% Others in train:  0.48754304233340084
% African-American in test:  0.5222672064777328
% Others in test:  0.47773279352226716
Difference in Recidivism Rate between Groups in Training Data:  0.14200462747815584
Difference in Recidivism Rate between Groups in Test Data:  0.13341216660097227


### 1.3 - Basic Model
Fit an out-of-the-box ```RandomForestClassifier``` from ```sklearn```. Use the ```predict``` method to return the model's predictions.

In [27]:
clf = RandomForestClassifier(random_state = 1000)
clf.fit(X_train, y_train)

#predictions on the testing set
y_hat = clf.predict(X_test)

### 1.4 Compute Metrics
Your task is to develop a function that:

* ***Takes as input***: the model's predictions on the test set, the actual observed outcomes on the test set, and the test features.
* ***Returns***: group-level accuracy, false-positive rate, precision, and recall metrics.

This function will be instrumental in assessing the fairness and effectiveness of our predictive model across different demographic groups.


In [28]:
def compute_metrics(y_hat,
                    y_test,
                    X_test):

    test = X_test.copy()
    test['outcome'] = y_test
    test['predicted'] = y_hat

   # Calculating metrics for group 0 (Others)
    TP_0 = ((test['predicted'] == 1) & (test['outcome'] == 1) & (test['groups'] == 0)).sum()
    FP_0 = ((test['predicted'] == 1) & (test['outcome'] == 0) & (test['groups'] == 0)).sum()
    TN_0 = ((test['predicted'] == 0) & (test['outcome'] == 0) & (test['groups'] == 0)).sum()
    FN_0 = ((test['predicted'] == 0) & (test['outcome'] == 1) & (test['groups'] == 0)).sum()

    # Calculating metrics for group 1 (African-American)
    TP_1 = ((test['predicted'] == 1) & (test['outcome'] == 1) & (test['groups'] == 1)).sum()
    FP_1 = ((test['predicted'] == 1) & (test['outcome'] == 0) & (test['groups'] == 1)).sum()
    TN_1 = ((test['predicted'] == 0) & (test['outcome'] == 0) & (test['groups'] == 1)).sum()
    FN_1 = ((test['predicted'] == 0) & (test['outcome'] == 1) & (test['groups'] == 1)).sum()

    # Calculate measures for g0
    accuracy_g0 = (TP_0 + TN_0) / (TP_0 + FP_0 + TN_0 + FN_0)
    precision_g0 = TP_0 / (TP_0 + FP_0) if (TP_0 + FP_0) > 0 else 0
    recall_g0 = TP_0 / (TP_0 + FN_0) if (TP_0 + FN_0) > 0 else 0
    false_positive_rate_g0 = FP_0 / (FP_0 + TN_0) if (FP_0 + TN_0) > 0 else 0

    # Calculate measures for g1
    accuracy_g1 = (TP_1 + TN_1) / (TP_1 + FP_1 + TN_1 + FN_1)
    precision_g1 = TP_1 / (TP_1 + FP_1) if (TP_1 + FP_1) > 0 else 0
    recall_g1 = TP_1 / (TP_1 + FN_1) if (TP_1 + FN_1) > 0 else 0
    false_positive_rate_g1 = FP_1 / (FP_1 + TN_1) if (FP_1 + TN_1) > 0 else 0



    metrics_df = pd.DataFrame({
    "Group": ["0-OtherRaces", "0-OtherRaces", "0-OtherRaces", "0-OtherRaces",
              "1-AfricanAmerican", "1-AfricanAmerican", "1-AfricanAmerican", "1-AfricanAmerican"],
    "Metric": ["Accuracy", "Precision", "Recall", "FPR", "Accuracy", "Precision", "Recall", "FPR"],
    "Value": [accuracy_g0, precision_g0, recall_g0, false_positive_rate_g0,
              accuracy_g1, precision_g1, recall_g1, false_positive_rate_g1]})

    return metrics_df

In [29]:
metrics_df = compute_metrics(y_hat = y_hat, y_test = y_test, X_test = X_test)

print(metrics_df.pivot(index='Metric', columns='Group', values='Value'))

Group      0-OtherRaces  1-AfricanAmerican
Metric                                    
Accuracy       0.649153           0.626357
FPR            0.252660           0.390769
Precision      0.517766           0.618619
Recall         0.476636           0.643750


In this assignment, we focus on ***disparate impact***, illustrated by differences in False Positive Rates (FPR) between African-Americans and other races. A higher FPR for African-Americans means more individuals from this group could be unjustly denied bail if such a model were used in decision-making. Although the baseline model may show higher accuracy, precision, and recall for African-Americans, it also results in a significantly higher FPR for this group. Our goal is to address and mitigate this bias, with a particular focus on reducing the disparate FPR.

## 2. Pre-processing for Bias Mitigation

Pre-processing is a strategic approach to mitigate bias, particularly effective when bias in the data may be attributed to how the data were sampled. In the context of our dataset, societal biases could influence lending decisions towards certain demographic groups. By addressing potential sampling biases, we aim to improve the fairness of our predictive model.

### 2.1 - Reweighting to Correct Sample Bias

We will use the approach discussed in class in which we re-weight training instances so that the algorithm learns more from smaller groups than it would in the case where every training instance is weighted equally. In this case, we will upweight training instances from smaller groups, setting the weight for the largest group equal to 1.

**Step 1: Identify the size of the most represented group within our dataset.**

In [30]:
group_counts = X_train['groups'].value_counts()
most_represented_group_count = np.max(group_counts)

**Step 2: Create a lambda function that will generate a weights column using this methodology**

In [31]:
X_train['weights'] = X_train['groups'].apply(lambda x: most_represented_group_count / group_counts[x])

**Step 3: Create a table showing that the training rows from different groups now have equal weight**

In [32]:
X_train.groupby('groups')['weights'].sum()

groups
0    2530.0
1    2530.0
Name: weights, dtype: float64

We see that this scheme has now equally weights both groups in the training data.

### 2.2 Fit, Predict, and Print the Weighted Model Results

With the dataset now reweighted to correct for sample bias, proceed to fit the model using this adjusted data. After fitting the model, predict the outcomes and evaluate the model's performance. Specifically, focus on how the reweighting has impacted the model's fairness across different demographic groups by examining changes in key performance metrics.

In [33]:
weights = X_train['weights']
X_train.drop(['weights'],axis=1,inplace=True)


clf_preprocess = RandomForestClassifier(random_state = 1000)
clf_preprocess.fit(X_train, y_train, sample_weight=weights)

#predictions on the testing set
y_hat_preprocess = clf_preprocess.predict(X_test)

metrics_df = compute_metrics(y_hat = y_hat_preprocess, y_test = y_test, X_test = X_test)

print(metrics_df.pivot(index='Metric', columns='Group', values='Value'))




Group      0-OtherRaces  1-AfricanAmerican
Metric                                    
Accuracy       0.650847           0.624806
FPR            0.247340           0.393846
Precision      0.520619           0.616766
Recall         0.471963           0.643750


We see that this approach did not reduce the FPR disparity between the groups. This is likely because the two groups are very similar in size in the data, thus sampling bias isn't a likely cause of the ***disparate impact*** we observe.

## 3. In-Processing for Bias Mitigation

In-processing is particularly effective for addressing bias stemming from ***differential subgroup validity***, where the relationship between predictors and outcomes may differ across groups. To tackle this, we will develop separate models for different demographic groups (in this case, by race). This approach allows the models to more accurately capture and reflect the unique relationships between features and outcomes within each subgroup.

### 3.1 - Split train and test data by group

Begin by segregating the dataset into separate training and testing sets by group. This division enables the creation of group-specific models, ensuring that each model is tailored to accurately represent its respective group.

In [34]:
def split_train_and_test_by_group(X_train, X_test, y_train, y_test):


    train = X_train.copy()
    test = X_test.copy()
    train['outcome'] = y_train
    test['outcome'] = y_test

    # split train and test by group
    train_g1 = train[train['groups']==1]
    train_g0 = train[train['groups']==0]
    test_g1 = test[test['groups']==1]
    test_g0 = test[test['groups']==0]

    # separate outcomes from test
    y_train_g1 = train_g1['outcome']
    y_train_g0 = train_g0['outcome']
    y_test_g1 = test_g1['outcome']
    y_test_g0 = test_g0['outcome']

    X_train_g1 = train_g1.drop(['outcome', 'groups'], axis=1)
    X_train_g0 = train_g0.drop(['outcome', 'groups'], axis=1)
    X_test_g1 = test_g1.drop(['outcome', 'groups'], axis=1)
    X_test_g0 = test_g0.drop(['outcome', 'groups'], axis=1)

    return y_train_g1, y_train_g0, y_test_g1, y_test_g0, X_train_g1, X_train_g0, X_test_g1, X_test_g0


y_train_g1, y_train_g0, y_test_g1, y_test_g0, \
X_train_g1, X_train_g0, X_test_g1, X_test_g0 = split_train_and_test_by_group(X_train = X_train,
                                                                              X_test = X_test,
                                                                              y_train = y_train,
                                                                              y_test = y_test)




### 3.2 - Fit models and generate predictions for each group

Now, fit separate predictive models for each group using the training data you've prepared. After fitting these models, generate predictions for each group-specific test set.

In [35]:
clf_g0 = RandomForestClassifier(random_state=1000)
clf_g0.fit(X_train_g0, y_train_g0)
clf_g1 = RandomForestClassifier(random_state=1000)
clf_g1.fit(X_train_g1, y_train_g1)
y_hat_g0 = clf_g0.predict(X_test_g0)
y_hat_g1 = clf_g1.predict(X_test_g1)

### 3.3 - Write a function to merge data back together and return a combined ```y_hat```, ```y_test```, and ```X_test```

After generating group-specific predictions, write a function that merges these predictions back into a single dataset. This function should return combined y_hat (predicted outcomes), y_test (actual outcomes), and X_test (test features) for the entire test dataset. This recombination is essential for evaluating the overall performance of our bias mitigation strategy across all groups.

In [36]:
def merge_separate_models(y_hat_g1, y_hat_g0, y_test_g1, \
                          y_test_g0, X_test_g1, X_test_g0):

    test_g1 = X_test_g1.copy()
    test_g1['groups'] = 1
    test_g1['outcome'] = y_test_g1
    test_g1['pred'] = y_hat_g1

    test_g0 = X_test_g0.copy()
    test_g0['groups'] = 0
    test_g0['outcome'] = y_test_g0
    test_g0['pred'] = y_hat_g0

    # Merge back together
    test = pd.concat([test_g0, test_g1])

    y_test = test['outcome']
    y_hat = test['pred']
    X_test = test.drop(['outcome','pred'], axis=1)

    return y_test, y_hat, X_test


y_test_combined, \
y_hat_combined, \
X_test_combined = merge_separate_models(y_hat_g1 = y_hat_g1,
                                        y_hat_g0 = y_hat_g0,
                                        y_test_g1 = y_test_g1,
                                        y_test_g0 = y_test_g0,
                                        X_test_g1 = X_test_g1,
                                        X_test_g0 = X_test_g0)






### 3.4 Print model output

In [37]:
metrics_df = compute_metrics(y_hat = y_hat_combined,
                             y_test = y_test_combined,
                             X_test = X_test_combined)

print(metrics_df.pivot(index='Metric', columns='Group', values='Value'))

Group      0-OtherRaces  1-AfricanAmerican
Metric                                    
Accuracy       0.654237           0.609302
FPR            0.220745           0.412308
Precision      0.528409           0.601190
Recall         0.434579           0.631250


We see that the approach does not improve the disparity in the FPR either. This points to the fact that **differential subgroup validity** may be less relevant in this case of bias, and if it is relevant, training classifiers that are group-specific does not address the problem.

## 4. Post-processing for Bias Mitigation.

This section introduces post-processing techniques as a means to further mitigate bias in our predictive models. Post-processing involves adjusting the model's output after the initial predictions have been made, offering another layer of bias correction. Sometimes, if there is significant bias in how outcomes are created (e.g. socially biased disparities in arrest rates amongst racial groups), post-processing may be the only method of correction.

### 4.1 Creating Different Thresholds for Different Groups

Until now, our models have applied the standard prediction threshold of 0.5, meaning a prediction label of 1 (positive) is assigned if the predicted probability exceeds this value. As an alternative, we'll explore optimizing prediction thresholds for each demographic group. Our goal is to enhance overall accuracy without infringing upon fairness principles, particularly avoiding disparate impact manifested through variations in False Positive Rates (FPR) between groups.

To achieve this, we will experiment with various threshold values above 0.5 (since 0.5 is our baseline). It's important to work with predicted probabilities rather than binary class labels, as this allows us to finely tune our threshold for decision-making.

In [38]:
threshold_values = [i/100 for i in range(50, 90)]
y_hat_probs = clf.predict_proba(X_test)[:,1]

Now, create a function that takes as input the true outcomes (y_true), the predicted probabilities (y_proba), and the list of threshold values previously created (threshold_values) and returns a dataframe that contains all possible combinations of threshold values for each group, as well as model accuracy and the difference in false positive rate.

In [39]:
def evaluate_group_thresholds(y_true, y_proba, groups, threshold_values):

    unique_groups = np.unique(groups)
    # This enumerates all possible options for threshold values within the range specified
    all_combinations = list(product(threshold_values, repeat=len(unique_groups)))

    # Prepare a list to collect results
    results = []

    for combination in all_combinations:
        group_thresholds = {group: threshold for group, threshold in zip(unique_groups, combination)}

        # Apply group-specific thresholds to generate predictions
        y_pred = np.zeros(y_true.shape)
        for group, threshold in group_thresholds.items():
            group_mask = (groups == group)
            y_pred[group_mask] = (y_proba[group_mask] > threshold).astype(int)

        # Calculate FPR for each group
        fprs = {}
        for group in unique_groups:
            group_mask = (groups == group)
            group_true = y_true[group_mask]
            group_pred = y_pred[group_mask]

            FP = len(group_true[(group_true == 0) & (group_pred == 1)])
            TN = len(group_true[(group_true == 0) & (group_pred == 0)])
            fprs[group] = FP / (FP + TN)

        # Calculate overall accuracy
        overall_accuracy = accuracy_score(y_true, y_pred)

        # Calculate the FPR difference
        fpr_diff = fprs[unique_groups[0]] - fprs[unique_groups[1]]
        # Create a row of data as a list of four items (see columns for below for ordering)
        row = list(combination) + [overall_accuracy, fpr_diff]
        results.append(row)

    # Create a DataFrame from the collected results
    columns = ['Threshold 0', 'Threshold 1', 'Model Accuracy', 'FPR Difference']
    results_df = pd.DataFrame(results, columns=columns)

    return results_df


results = evaluate_group_thresholds(y_true = y_test,
                                    y_proba = y_hat_probs,
                                    groups = X_test['groups'],
                                    threshold_values = threshold_values)

Now that you have evaluated many combinations of thresholds, lets see where the thresholds fall below a **0.01** difference in false positive rate, and let's pick the row in this range with the highest model accuracy.

In [40]:
# Filter the DataFrame for rows where the absolute FPR difference is less than 0.01
fpr_threshold_subset = results[results['FPR Difference'].abs()<0.01]

# Find the index of the row with the maximum accuracy in the filtered subset
max_accuracy_index = fpr_threshold_subset['Model Accuracy'].idxmax()

# Retrieve the row with the maximum accuracy
max_accuracy_row = fpr_threshold_subset.loc[max_accuracy_index]
print(max_accuracy_row)

Threshold 0       0.570000
Threshold 1       0.710000
Model Accuracy    0.650202
FPR Difference   -0.000270
Name: 301, dtype: float64


Finally, let's apply these chosen thresholds to our test-set predictions, and then calculate our standard set of metrics.

In [41]:
def generate_group_level_labels(y_proba, groups, max_accuracy_row):

    y_pred = np.zeros(y_proba.shape)
    for group in groups.unique():
        group_mask = (groups == group)
        y_pred[group_mask] = (y_proba[group_mask] > max_accuracy_row[f'Threshold {group}']).astype(int)

    return y_pred


y_hat_thresholds = generate_group_level_labels(y_proba = y_hat_probs,
                                               groups = X_test['groups'],
                                               max_accuracy_row = max_accuracy_row)



In [42]:
metrics_df = compute_metrics(y_hat = y_hat_thresholds,
                             y_test = y_test,
                             X_test = X_test)

print(metrics_df.pivot(index='Metric', columns='Group', values='Value'))

Group      0-OtherRaces  1-AfricanAmerican
Metric                                    
Accuracy       0.674576           0.627907
FPR            0.178191           0.178462
Precision      0.570513           0.704082
Recall         0.415888           0.431250


Well done! We see that this post-processing technique has significantly reduced the disparity in False Positive Rate! However, this may be a case of **disparate treatment**, as there are now effectively different decision rules for different racial groups.